<div class="jumbotron">
    <center>
        <h1>The last mile</h1>
        <h2>From Spark Streaming to Visualization in 10 steps</h2>
    </center>
</div>

![](https://i.pinimg.com/564x/85/88/9a/85889a585974c7331b10343c98a53ee8.jpg)
[Pinterest](https://www.pinterest.ca/pin/255649716331177273/)

 # 1. from spark to elastic search

![](https://static-www.elastic.co/v3/assets/bltefdd0b53724fa2ce/blt7f9ca77d747fb53d/5c240aa34cca137b3874384e/es-hadoop-diagram.svg)

# Doc please
* https://www.elastic.co/what-is/elasticsearch-hadoop
* https://www.elastic.co/guide/en/elasticsearch/hadoop/current/spark.html
* https://www.elastic.co/guide/en/elasticsearch/hadoop/8.2/install.html

![](https://codinginfinite.com/wp-content/uploads/2019/05/documentation-where-were-going-we-dont-need-documentation.jpg)
[Source](https://codinginfinite.com/2019-programming-funny-jokes-memes/)

Ok quite simple first we need a driver to add, we can add to packages to be passed to spark-submit
and maven will do to do the rest

> The Spark connector framework is the most sensitive to version incompatibilities. For your convenience, a version compatibility matrix has been provided below:

```bash
bash sparkSubmitPython.sh twitter_stream_elastic.py "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1,org.elasticsearch:elasticsearch-spark-30_2.12:8.2.0"
```

# 2. Define the index

Altough Elastic Search do automatically index creation and guess on field, 
sometimes is better to predefine the index and define mapping

This requires add elasticsearch python module to Spark docker image

```Dockerfile 
RUN pip install pyspark numpy elasticsearch
``` 

Then we need to create the mapping

#### Credit for mapping https://medium.com/@CMpoi/elasticsearch-defining-the-mapping-of-twitter-data-dafad0f50695 Timestamp
```python
es_mapping = {
    "mappings": {
        "properties": 
            {
                "created_at": {"type": "date","format": "EEE MMM dd HH:mm:ss Z yyyy"},
                "text": {"type": "text","fielddata": True}
            }
    }
}
```

Then connect to Elasticsearch and create the index

```python
es = Elasticsearch(hosts=elastic_host) 
# make an API call to the Elasticsearch cluster
# and have it return a response:
response = es.indices.create(
    index=elastic_index,
    body=es_mapping,
    ignore=400 # ignore 400 already exists code
)

if 'acknowledged' in response:
    if response['acknowledged'] == True:
        print ("INDEX MAPPING SUCCESS FOR INDEX:", response['index'])
```

![](https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTMTXhs9fd7M_qo4XA_6OiKLX9GaTldJWPICDtIGCB7SuE6q9NM&usqp=CAU)

# 3. Define Schema

```python
tweetKafka = tp.StructType([
    tp.StructField(name= 'id_str', dataType= tp.StringType(),  nullable= True),
    tp.StructField(name= 'created_at', dataType= tp.StringType(),  nullable= True),
    tp.StructField(name= 'text',       dataType= tp.StringType(),  nullable= True)
])
```

![](https://www.memecreator.org/static/images/memes/4772258.jpg)

# 4. Configure Elastic Search Parameters

```python
elastic_host="10.0.100.51"
elastic_index="tapwteet"
elastic_document="_doc"
}
```

![](http://www.quickmeme.com/img/6d/6d9b3966c39d711a732a842f41bc39f7023eb4fafdc9ecb22d0f8ca1c2fee05e.jpg)
[Source](http://www.quickmeme.com/meme/3tvnt7)

# 5. Take data

```python
   data2.select("id_str", "created_at", "text", "prediction") \
```

![](https://i.imgflip.com/3ddyn4.jpg)
[Source](https://imgflip.com/i/3ddyn4)

# 6. Send to ES

```python
 # Write the stream to elasticsearch
 df.writeStream \
    .option("checkpointLocation", "/save/location") \
    .format("es") \
    .start(elastic_index) \
 ```

![](https://i.imgflip.com/373hpn.jpg)
[Source](https://imgflip.com/i/373hpn)

# 8. Run

```bash
bash kafkaStartZk.sh
bash kafkaStartServer.sh
bash flumeTwitterKafka.sh || logstashTwitterKafka.sh
bash elasticSearch.sh
bash sparkSubmitPython.sh twitter_stream_elastic.py "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1,org.elasticsearch:elasticsearch-spark-30_2.12:8.2.0"

bash sparkSubmitPython.sh twitter_stream_elastic_nml.py "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1,org.elasticsearch:elasticsearch-spark-30_2.12:8.2.0"
```

![](https://i.chzbgr.com/full/2922317568/h5AC86B91/but-wait-theres-more)

# 9. Define Index Pattern in  Kibana

# 10. start creating visualizations and search

# Biblio

* https://www.elastic.co/what-is/elasticsearch-hadoop
* https://www.elastic.co/guide/en/elasticsearch/hadoop/current/spark.html
* https://medium.com/@akkidx/indexing-into-elasticsearch-using-spark-code-snippets-55eabc753272
* https://github.com/vinayak0792/SparkStreaming/blob/master/consumer.py